In [1]:
import pandas as pd
import numpy as np

# Data Cleaning and Rough Draft for ACFT Calculaor 

## I took excel tables from here: https://companyleader.themilitaryleader.com/2019/11/07/acft-calculator-tracker/
## That are accurate to the US Army PDF that's more difficult to take data from here: https://www.goarmy.com/how-to-join/requirements/fitness

### Being new to Swift, I chose to write a rough draft version of the main calculations in Python. Knowing that Swift seems to favor funciotnal programming I went with dictionaries over Python objects. This also made sense with the end goal being to place the data in JSON (since JavaScript objects are pretty similar to Python dictionaries)

### I felt the author, "The Company Leader", of the Excel calculator was too faithful to the Army website's formatting. I'm taking his excel tables, turning them to pandas data frames, subsetting them as needed to make the JSON file for the actual app. There's also a function to do the primary logic of the iOS app that takes a dictionary as an argument and calculate an indivual's score. 

### There are three stages to this app: (1) basic iOS ACFT calculator for an individual, (2) add SQLite for tracking individual historic performance metrics and for unit leaders to quickly log results for their soldiers and recieve an Excel document with the results, and (3) create a feature to scan handwriting on DA 705s to instantly get ACFT scores.  This notebook is only relevant for stage one of this app. 

## converting excel tables to data frames to be cleaned for each event 

 ### copied and pasted score charts into my own excel file and creating data frames from there 

In [2]:
# male deadlift data 

dl_data = [
    [17, 22, 27, 32, 37, 42, 47, 52, 57, 62, 'Points'],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 0],
    [90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 10],
    [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 20],
    [110, 110, 110, 110, 110, 110, 110, 110, 110, 110, 30],
    [120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 40],
    [130, 130, 130, 130, 130, 130, 130, 130, 130, 130, 50],
    [140, 140, 140, 140, 140, 140, 140, 140, 140, 140, 60],
    [150, 150, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 61],
    [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 62],
    [np.nan, 150, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 63],
    [160, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 64],
    [np.nan, 160, 160, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 65],
    [np.nan, np.nan, np.nan, 150, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 66],
    [170, 170, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 67],
    [180, np.nan, 170, 160, 150, np.nan, np.nan, np.nan, np.nan, np.nan, 68],
    [190, 180, 180, 170, np.nan, 150, np.nan, np.nan, np.nan, np.nan, 69],
    [np.nan, 190, 190, 180, 160, np.nan, 150, 150, np.nan, np.nan, 70],
    [np.nan, 200, 200, 190, 170, np.nan, np.nan, np.nan, 150, np.nan, 71],
    [200, np.nan, np.nan, np.nan, 180, np.nan, np.nan, np.nan, np.nan, 150, 72],
    [np.nan, 210, 210, 200, np.nan, 160, np.nan, np.nan, np.nan, np.nan, 73],
    [210, np.nan, np.nan, np.nan, 190, np.nan, np.nan, np.nan, np.nan, np.nan, 74],
    [np.nan, 220, 220, 210, np.nan, 170, np.nan, np.nan, np.nan, np.nan, 75],
    [np.nan, np.nan, np.nan, np.nan, 200, 180, np.nan, np.nan, np.nan, np.nan, 76],
    [220, 230, 230, 220, np.nan, 190, np.nan, np.nan, np.nan, np.nan, 77],
    [np.nan, np.nan, np.nan, np.nan, 210, np.nan, 160, np.nan, np.nan, np.nan, 78],
    [230, 240, 240, 230, np.nan, 200, np.nan, 160, 160, np.nan, 79],
    [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 170, np.nan, np.nan, np.nan, 80],
    [np.nan, 250, 250, 240, 220, 210, 180, np.nan, np.nan, np.nan, 81],
    [240, np.nan, np.nan, np.nan, 230, np.nan, np.nan, np.nan, np.nan, 160, 82],
    [250, np.nan, np.nan, 250, np.nan, np.nan, 190, np.nan, np.nan, np.nan, 83],
    [260, 260, 260, np.nan, 240, 220, np.nan, 170, np.nan, np.nan, 84],
    [np.nan, np.nan, 270, 260, np.nan, 230, 200, 180, np.nan, np.nan, 85],
    [np.nan, 270, np.nan, np.nan, 250, np.nan, np.nan, 190, np.nan, np.nan, 86],
    [270, np.nan, 280, 270, np.nan, 240, 210, np.nan, np.nan, np.nan, 87],
    [np.nan, 280, 290, 280, 260, np.nan, np.nan, 200, np.nan, np.nan, 88],
    [np.nan, 290, 300, 290, np.nan, 250, 220, np.nan, 170, np.nan, 89],
    [280, 310, 310, 300, 270, np.nan, 230, 210, np.nan, np.nan, 90],
    [290, np.nan, 320, 310, 280, 260, 240, np.nan, 180, np.nan, 91],
    [np.nan, 320, np.nan, np.nan, 290, 270, 250, np.nan, np.nan, 170, 92],
    [300, np.nan, np.nan, 320, 300, 280, 260, 220, 190, 180, 93],
    [np.nan, np.nan, np.nan, np.nan, 310, 290, 270, 230, 200, 190, 94],
    [310, np.nan, np.nan, np.nan, 320, 300, 280, 240, np.nan, 200, 95],
    [np.nan, np.nan, np.nan, np.nan, np.nan, 310, 290, 250, 210, np.nan, 96],
    [330, np.nan, np.nan, np.nan, np.nan, 320, 300, 260, 220, np.nan, 97],
    [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 310, 270, 230, 210, 98],
    [330, 330, 330, 330, 330, 330, 320, 280, 240, 220, 99],
    [340, 340, 340, 340, 340, 340, 330, 290, 250, 230, 100]
]

male_deadlift = pd.DataFrame(dl_data[1:], columns=dl_data[0])




male_deadlift

,17,22,27,32,37,42,47,52,57,62,Points
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,0
2,90.0,90.0,90.0,90.0,90.0,90.0,90.0,90.0,90.0,90.0,10
3,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,20
4,110.0,110.0,110.0,110.0,110.0,110.0,110.0,110.0,110.0,110.0,30
5,120.0,120.0,120.0,120.0,120.0,120.0,120.0,120.0,120.0,120.0,40
6,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,50
7,140.0,140.0,140.0,140.0,140.0,140.0,140.0,140.0,140.0,140.0,60
8,150.0,150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62


In [16]:
# importing each sheet as a data frame 

# male scoring charts 
male_dl = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_dl").replace('---', np.nan)
male_spt = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_spt").replace('---', np.nan)
male_hrp = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_hrp").replace('---', np.nan)
male_sdc = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_sdc").replace('---', np.nan)
male_plk = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_plk").replace('---', np.nan)
male_tmr = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_tmr").replace('---', np.nan)

# female scoring charts
female_dl = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_dl").replace('---', np.nan)
female_spt = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_spt").replace('---', np.nan)
female_hrp = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_hrp").replace('---', np.nan)
female_sdc = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_sdc").replace('---', np.nan)
female_plk = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_plk").replace('---', np.nan)
female_tmr = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_tmr").replace('---', np.nan)

# we'll work with these later
# They're the alternate cario events for pass/fail 


# male alternative cardio 
male_walk = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_walk")
male_swim = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_swim")
male_bike = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_bike")
male_row = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_row")
# female alternative cardio
female_walk = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_walk")
female_swim = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_swim")
female_bike = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_bike")
female_row = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_row")


# col names are all strings for some reason... fixing it 

usable_col_names = [17, 22, 27, 32, 37, 42, 47, 55, 57, 62, 'Points'] # making the nums ints not strings 


sheets = [ # list of all data frames 
    male_dl, male_spt, male_hrp, male_sdc, male_plk, male_tmr,
    female_dl, female_spt, female_hrp, female_sdc, female_plk, female_tmr,
    male_walk, male_swim, male_bike, male_row,
    female_walk, female_swim, female_bike, female_row
]

for df in sheets:  
    df.columns = usable_col_names

female_dl.columns


Index([17, 22, 27, 32, 37, 42, 47, 55, 57, 62, 'Points'], dtype='object')

In [19]:
def get_acft_score(soldier_data): 
    """Expects a dictionary of information to calculate the ACFT Score. For now it's only a proof of concept for deadlift scores. 
    male_deadlift is temporarily a global variable. Upon realization that Swift uses FP and OOP, I'll might use an object over dictionary"""
    age_range = [62, 57, 52, 47, 42, 37, 32, 27, 22, 17] # decending min age for each age scale, constant 
   
   # determine which age scale to use 
    if soldier_data['age'] >= 62: 
        age_scale = 62
    else: 
        i = 0 
        while soldier_data['age'] < age_range[i]: 
            i += 1
            age_scale = age_range[i]

    # creating subsets of points and events..maybe a more numpy way of doing this, but this works
    dl_weights = male_dl[age_scale].tolist()
    points = male_dl['Points'].tolist()

    # series of simple for loops to match the individual soldier's performance 
    # on each event with correct points 
    for j in range(len(dl_weights)): 
        if dl_weights[j] != 'nan' and soldier_data['deadlift'] >= dl_weights[j]: 
            dl_index = j
    
    

    return points[dl_index]
            

# example dictionary 

my_score = {
    'age': 30, 
    'gender': 'male', 
    'deadlift' : 320
}

get_acft_score(my_score)



91

In [18]:
male_dl.columns 

Index([17, 22, 27, 32, 37, 42, 47, 55, 57, 62, 'Points'], dtype='object')

In [17]:
male_deadlift.columns

Index([17, 22, 27, 32, 37, 42, 47, 52, 57, 62, 'Points'], dtype='object')